### Imports & EC3 Python Wrapper Setup

In [1]:
import os
import pandas as pd
import json
from ec3 import EC3epds #Requires pip install ec3-python-wrapper

token = os.environ['EC3_KEY'] #assumes EC3 access token is stored as environment variable
ec3_epds = EC3epds(bearer_token=token, ssl_verify=False)

### Main EC3 Query
**This will take some time to run!**

In [93]:
#TIMES: 5,000 psi TOOK ~13 MIN TO RUN
#TIMES: 4,000 psi TOOK ~19 MIN TO RUN
#TIMES: 3,000 psi TOOK ~9 MIN TO RUN

epd_param_dict = {"product_classes":{"EC3":"Concrete >> ReadyMix"}, 
                  "concrete_compressive_strength_28d":"4000 psi",
                  "lightweight":False,
                  "plant_geography": ["US"],
                  "product_specific": True,
                  "manufacturer_specific": True
                  }

ec3_epds.only_valid = False

ec3_epds.return_fields = ["id", "date_of_issue", "concrete_compressive_strength_28d", "gwp", "gwp_per_category_declared_unit", "plant_geography", "lightweight", "plant_or_group"]
ec3_epds.sort_by = "date_of_issue"

epd_records = ec3_epds.get_epds(return_all=True, params=epd_param_dict)

In [94]:
#check how many EPDs were returned
print(len(epd_records))

23069


### Save to JSON / Load JSON

In [95]:
#While not required, it is likely helpful to save the EPD records to a json file for future use so that you can skip the above step
with open('epddata_4000psi.json', 'w') as f:
    json.dump(epd_records, f)

In [5]:
### Start from here once JSON file is saved ###

import json

with open('epddata_4000psi.json', 'r') as f:
    epd_list_4000 = json.load(f)

### Store just what we need

In [6]:
def convert_epd_list(epd_list):
    filt_epd_list = []
    for epd in epd_list:
        new_dict = {}
        new_dict['id'] = epd['id']
        new_dict['date_of_issue'] = epd['date_of_issue']
        new_dict['compressive_strength'] = epd.get('concrete_compressive_strength_28d')
        new_dict['gwp'] = epd.get('gwp')
        new_dict['gwp_per_category_declared_unit'] = epd.get('gwp_per_category_declared_unit')
        new_dict['plant_geography'] = epd.get('plant_geography')
        new_dict['plant_country'] =  epd.get('plant_or_group', {}).get('country')
        new_dict['plant_subdiv'] = epd.get('plant_or_group', {}).get('admin_district')

        filt_epd_list.append(new_dict)
        
    return filt_epd_list

In [7]:
filt_epd_list_4000 = convert_epd_list(epd_list_4000)

### Cleaning of the Data

In [8]:
# convert to dataframe and drop rows missing critical values
df_4ksi = pd.DataFrame(filt_epd_list_4000)
df_4ksi.dropna(subset=['compressive_strength', 'gwp', 'plant_country'], inplace=True)
df_4ksi = df_4ksi[df_4ksi['plant_country'] == 'US'] #extra filtering for plant country (may not be needed for all mixes)

In [9]:
#add a column for the gwp formatted as a float converted to kgCO2e per cubic yard
df_4ksi['gwp_val'] = df_4ksi['gwp_per_category_declared_unit'].str.extract('(\d+\.?\d*)').astype(float).round(1) * 0.764555

In [10]:
def remove_outliers(df, col_names):
    """
    Remove extreme outliers based on IQR method
    """
    q1 = df[col_names].quantile(0.25)
    q3 = df[col_names].quantile(0.75)
    iqr = q3 - q1

    df = df[~((df[col_names] < (q1 - 1.5 * iqr)) |(df[col_names] > (q3 + 1.5 * iqr))).any(axis=1)]

    return df

In [11]:
df_4ksi = remove_outliers(df_4ksi, ['gwp_val'])

In [12]:
#add a column for the date formatted as a datetime object
df_4ksi['date_formatted'] = pd.to_datetime(df_4ksi['date_of_issue'], format='%Y-%m-%d')

df_4ksi = df_4ksi[df_4ksi['date_formatted'].dt.year >= 2018]
len(df_4ksi) #this will be the length of the main dataset moving forward

32063

In [13]:
CLF_BASELINE_4000N = 235.6 #CLF baseline value in kgCO2e per cubic yard

In [14]:
# Filter the dataframe to only include rows where the year is 2023
df_2023 = df_4ksi[df_4ksi['date_formatted'].dt.year == 2023]

# Calculate the average of the 'gwp' values in the filtered dataframe
ec3_avg_23 = int(df_2023['gwp_val'].mean())

# Calculate the percentage of rows where 'gwp_val' is less than 235.6 (CLF baseline)
percentage_23 = (df_2023['gwp_val'] < CLF_BASELINE_4000N).mean() * 100

print(ec3_avg_23)
print(percentage_23) #percentage of values below CLF baseline

266
31.07116654438738


### Imports / Setup for Plotly

In [22]:
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go

pio.templates[pio.templates.default].layout.colorway = px.colors.diverging.Tealrose

### Plot #1

In [29]:

fig = go.Figure()

# Add scatter trace with medium sized markers
fig.add_trace(
    go.Scatter(
        mode='markers',
        x=df_4ksi['date_of_issue'],
        y=df_4ksi['gwp_val'],
        marker=dict(
            color='rgba(0,147,146,0.3)',
            size=5
        )
    ))

# Add lines and annotations to show average and baseline values
fig.add_shape(type='line', y0=CLF_BASELINE_4000N, y1=CLF_BASELINE_4000N, xref='paper', x0=0, x1=1, #4000 psi concrete
              line=dict(dash='dot'), 
              line_color="black",
              line_width=2)
fig.add_shape(type='line', y0=ec3_avg_23, y1=ec3_avg_23, xref='paper', x0=0, x1=1, 
              line=dict(dash='dot'), 
              line_color="dark grey",
              line_width=2)

fig.add_annotation(xref='paper', x=0.0, y=CLF_BASELINE_4000N, text="<b>2023 CLF Baseline<b>", bgcolor='rgba(255, 255, 255, 0.7)', showarrow=False, yshift=-10) #4000 psi concrete
fig.add_annotation(xref='paper', x=0.0, y=ec3_avg_23, text="<b>2023 EC3 Average<b>", bgcolor='rgba(255, 255, 255, 0.7)', showarrow=False, yshift=10)

# Update axes, title, etc.
fig.update_layout(
    title="Reported GWP of EPDs for 4000 psi Normal Weight Concrete Over Time",
    xaxis_title="Date EPD Issued",
    yaxis_title="GWP (kgCO2e/yd³)"
)

fig.show()

### Plot #2

In [30]:
# Group the rows by date and state and count the number of rows for each group
df_count = df_4ksi.groupby(['date_formatted', 'plant_subdiv']).size().reset_index(name='count')

# Pivot the dataframe to create a matrix of counts by date and state
df_pivot = df_count.pivot(index='date_formatted', columns='plant_subdiv', values='count').fillna(0)

# Sort the columns by the total number of rows in descending order
df_pivot = df_pivot[df_pivot.sum().sort_values(ascending=False).index]

# Create a stacked area chart using Plotly Graph Objects
fig2 = go.Figure()

for col in df_pivot.columns:
    fig2.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot[col].cumsum(), mode='lines', stackgroup='one', name=col))


# Add a title and axis labels to the chart
# Sort the legend by the total number of rows for each state and add the percentage
fig2.update_layout(legend=dict(title='State', orientation='v', yanchor='top', y=1.0, xanchor='right', x=1.1,
                itemsizing='constant', itemclick='toggleothers', traceorder='normal'), plot_bgcolor='white',
                title='Issued 4000 PSI NWC EPDs over Time by State (2018-present)', xaxis_title='Date', yaxis_title='Count of EPDs Issued')

fig2.update_yaxes(
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)

fig2.update_xaxes(
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)

# Calculate the percentage of each state and add it to the legend
for i, trace in enumerate(fig2.data):
    state = trace.name
    total_rows = df_pivot[state].sum()
    percentage = f'{total_rows / df_pivot.sum().sum() * 100:.2f}%'
    fig2.data[i].name = f'{state} ({percentage})'

# Show the chart
fig2.show()

### Plot #3

In [31]:
def get_national_avg_per_yr(df):
    # Group the rows by year and calculate the average of the 'gwp_val' column for each group
    df_avg_national = df.groupby(df['date_formatted'].dt.year)['gwp_val'].agg(['mean', 'count']).reset_index()
    df_avg_national.insert(1, 'region', 'USA')
    # Rename the columns of the new dataframe
    df_avg_national.columns = ['year', 'region', 'avg_gwp_val', 'row_count']

    return df_avg_national

In [32]:
df_avg_national_4ksi = get_national_avg_per_yr(df_4ksi)

In [33]:
# Group the rows by year and state and calculate the average gwp_val for each group
df_avg2 = df_4ksi.groupby([df_4ksi['date_formatted'].dt.year, 'plant_subdiv'])['gwp_val'].agg(['mean', 'count']).reset_index()
df_avg2.columns = ['year', 'region', 'avg_gwp_val', 'row_count']

plant_subdiv_list = ['CA', 'NJ', 'WA', 'OR', 'AZ', 'CO'] #4000 psi leaders by EPD count (minus MN)

# Filter the rows of the df_avg2 dataframe to only include rows with row_count > 30 and where state is in the plant_subdiv_list
df_avg_regions = df_avg2[(df_avg2['row_count'] >= 30) & (df_avg2['region'].isin(plant_subdiv_list))]

In [34]:
# Define a custom color sequence that excludes yellow
custom_color_sequence = px.colors.diverging.Tealrose[:3] + px.colors.diverging.Tealrose[4:]

# Create a Plotly line chart for the state-by-state averages
fig4 = go.Figure()

for i, region in enumerate(plant_subdiv_list):
    df_region = df_avg_regions[df_avg_regions['region'] == region]
    fig4.add_trace(go.Scatter(x=df_region['year'], 
                              y=df_region['avg_gwp_val'], 
                              mode='lines+markers', 
                              name=region, 
                              line=dict(width=2, dash='dot', color=custom_color_sequence[i])))

# Create a Plotly line chart for the national averages
fig4.add_trace(go.Scatter(x=df_avg_national_4ksi['year'], 
                          y=df_avg_national_4ksi['avg_gwp_val'], 
                          mode='lines+markers', name='USA', 
                          line=dict(width=3, color=px.colors.sequential.Blugrn[len(plant_subdiv_list)])))

# Add a title and axis labels to the chart
fig4.update_layout(title='Average GWP Value over Time', 
                   xaxis_title='Year', 
                   yaxis_title='Average GWP Value [kgCO2e/yd³]',
                   plot_bgcolor='white'
                   )

fig4.update_yaxes(
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)

fig4.update_xaxes(
    showline=True,
    linecolor='lightgrey',
    gridcolor='lightgrey'
)


# Show the chart
fig4.show()